<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/ComplexDatatypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,003 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd

In [21]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col


In [14]:
df_array = spark.sql("SELECT array(1, 2, 3) as my_array")
df_array.show()

+---------+
| my_array|
+---------+
|[1, 2, 3]|
+---------+



In [16]:
spark.sql("SELECT struct(1, 2, 3) as ex_struct")

DataFrame[ex_struct: struct<col1:int,col2:int,col3:int>]

In [34]:
df_struct = spark.sql("SELECT struct(1, 2, '3') as ex_struct")
df_struct.show()

+---------+
|ex_struct|
+---------+
|{1, 2, 3}|
+---------+



In [36]:
df_struct.select("ex_struct.col3", "ex_struct").show()

+----+---------+
|col3|ex_struct|
+----+---------+
|   3|{1, 2, 3}|
+----+---------+



In [37]:
df_struct.select("ex_struct.*").show(truncate=False)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|1   |2   |3   |
+----+----+----+



In [26]:
df_map = spark.sql("SELECT map(1.0, '2', 3.0, '4') as ex_map")
df_map.show()

+--------------------+
|              ex_map|
+--------------------+
|{1.0 -> 2, 3.0 -> 4}|
+--------------------+



In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# Data
data = [
        (("James",None,"Smith"),"OH","M"),
        (("Anna","Rose",""),"NY","F"),
        (("Julia","","Williams"),"OH","F"),
        (("Maria","Anne","Jones"),"NY","M"),
        (("Jen","Mary","Brown"),"NY","M"),
        (("Mike","Mary","Williams"),"OH","M")
        ]

# Schema
schema = StructType([
    StructField('name', StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
         ])),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True)
     ])

# Create DataFrame
df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False) # shows all columns

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+-----+------+
|name                  |state|gender|
+----------------------+-----+------+
|{James, NULL, Smith}  |OH   |M     |
|{Anna, Rose, }        |NY   |F     |
|{Julia, , Williams}   |OH   |F     |
|{Maria, Anne, Jones}  |NY   |M     |
|{Jen, Mary, Brown}    |NY   |M     |
|{Mike, Mary, Williams}|OH   |M     |
+----------------------+-----+------+



In [28]:
df2.select("name").show()

+--------------------+
|                name|
+--------------------+
|{James, NULL, Smith}|
|      {Anna, Rose, }|
| {Julia, , Williams}|
|{Maria, Anne, Jones}|
|  {Jen, Mary, Brown}|
|{Mike, Mary, Will...|
+--------------------+



In [113]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType
from pyspark.sql.functions import element_at

data = [
        ([("Yash","K",None),("Yash2","K2",None)],"BL","M",["KGF 1", "KGF 2"],{"Bank":"100 CR", "Business":"50 CR", "Land":"150 CR"}),
        (("Sudeep","Kicha","S"),"DL","M",["Autograph", "Kicha","Hucha"],{"Others":"300 CR"}),
        (("Puneeth",None,"Raj"),"MB","M",[], {"Building":"500 CR", "Commercal":100}),
        (("Darshan",None,None),"MB","M",[], {"Building":"500 CR", "Commercal":100}),

        ]

# Schema
schema = StructType([
    StructField(    'name'
                  , ArrayType(
                        StructType([
                            StructField('firstname', StringType(), True),
                            StructField('middlename', StringType(), True),
                            StructField('lastname', StringType(), True)
                        ])
                    ), True),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True),
     StructField('movies', ArrayType(StringType()), True),
     StructField('properties', MapType(StringType(),StringType()), True)
     ])

# Create DataFrame
df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False) # shows all columns

PySparkTypeError: [CANNOT_ACCEPT_OBJECT_IN_TYPE] `StructType` can not accept object `Sudeep` in type `str`.

In [110]:
 (df2
  .select("properties", "name", "name.firstname")
  .withColumn("M-Building", col("properties").getItem("Building"))
  .withColumn("M-Commercal ", col("properties").getItem("Commercal"))
  .withColumn("M-Others ", col("properties").getItem("Others"))
  .withColumn("S-FName", col("name.firstname"))
  ).show()

+--------------------+--------------------+---------+----------+------------+---------+-------+
|          properties|                name|firstname|M-Building|M-Commercal |M-Others |S-FName|
+--------------------+--------------------+---------+----------+------------+---------+-------+
|{Bank -> 100 CR, ...|     {Yash, K, NULL}|     Yash|      NULL|        NULL|     NULL|   Yash|
|  {Others -> 300 CR}|  {Sudeep, Kicha, S}|   Sudeep|      NULL|        NULL|   300 CR| Sudeep|
|{Building -> 500 ...|{Puneeth, NULL, Raj}|  Puneeth|    500 CR|         100|     NULL|Puneeth|
|{Building -> 500 ...|{Darshan, NULL, N...|  Darshan|    500 CR|         100|     NULL|Darshan|
+--------------------+--------------------+---------+----------+------------+---------+-------+



In [99]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType
from pyspark.sql.functions import element_at

data = [
        ([("Yash","K",None),],"BL","M",["KGF 1", "KGF 2"],{"Bank":"100 CR", "Business":"50 CR", "Land":"150 CR"}),
        (("Sudeep","Kicha","S"),"DL","M",["Autograph", "Kicha","Hucha"],{"Others":"300 CR"}),
        (("Puneeth",None,"Raj"),"MB","M",[], {"Building":"500 CR", "Commercal":100}),
        (("Darshan",None,None),"MB","M",[], {"Building":"500 CR", "Commercal":100}),

        ]

# Schema
schema = StructType([
    StructField('name', StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
         ])),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True),
     StructField('movies', ArrayType(StringType()), True),
     StructField('properties', MapType(StringType(),StringType()), True)
     ])

# Create DataFrame
df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False) # shows all columns

PySparkValueError: [LENGTH_SHOULD_BE_THE_SAME] obj and fields should be of the same length, got 1 and 3.